In [ ]:
#imports
#from google.colab import drive
from scipy.io import loadmat
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np
import math
import os
import time
import datetime
import itertools
import h5py
import matplotlib.dates as mdates


# define the data location and load data
cwd = os.getcwd()
pardir = os.path.dirname(cwd)

#### commented for google drive
#drive.mount('/content/drive')
#pardir = "/content/drive/MyDrive/Stanford-solar-forecasting-dataset/"


data_folder = os.path.join(pardir,"data")#,"data_forecast")
data_path = os.path.join(data_folder, "2017_2019_images_pv_processed_forecast.hdf5") #forecast_dataset
output_folder = oc.path.join(pardir, "model_output/SUNSET_forecast/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import data


In [ ]:
times_trainval = np.load(data_dir+"times_trainval.npy", allow_pickle=True)
f = h5py.File(data_dir+'2017_2019_images_pv_processed_forecast.hdf5', 'r')
test = f['test']
pv_log = test['pv_log']
images_log = test['images_log']
pv_log = pv_log[:]

# get the input dimension for constructing the model
img_side_len = f['trainval']['images_log'].shape[1]
num_color_channel = f['trainval']['images_log'].shape[3]
image_input_dim = [img_side_len,img_side_len,num_color_channel]
#MYEDIT: getting the dim of the sun_pos
sun_pos_dim = f['trainval']['sun_pos'].shape[1]
sampi_dim = f['trainval']['sampi'].shape[1]

print("image side length:", img_side_len)
print("number of color channels:", num_color_channel)
print("input image dimension:", image_input_dim)
#MYEDIT
print("sun_pos dimension:", sun_pos_dim)
print("sampi dimension:", sampi_dim)


f.close()

image side length: 64
number of color channels: 3
input image dimension: [64, 64, 3]
sun_pos dimension: 2
sampi dimension: 1


### Model architecture

In [ ]:
# define model characteristics
num_filters = 24
kernel_size = [3,3]
pool_size = [2,2]
strides = 2
dense_size = 1024
drop_rate = 0.4

# define training time parameters
num_epochs = 200 #(The maximum epoches set to 200 and there might be early stopping depends on validation loss)
num_fold = 10 # 10-fold cross-validation
batch_size = 256
learning_rate = 3e-06

In [ ]:
# define the model architecture using tf.keras API
def sunset_model():
    ## input
    ### input image logs with shape (64,64,3)
    x_in = keras.Input(shape=image_input_dim)
    ###MYEDIT: input of the sun position with shape (2) and sampi with shape (1)
    x2_in = keras.Input(shape=sun_pos_dim)
    x3_in = keras.Input(shape=sampi_dim)

    ## 1st convolution block
    x = keras.layers.Conv2D(num_filters,kernel_size,padding="same",activation='relu')(x_in)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D(pool_size, strides)(x)

    ## 2nd convolution block
    x = keras.layers.Conv2D(num_filters*2,kernel_size,padding="same",activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D(pool_size, strides)(x)

    ## two fully connected nets
    x = keras.layers.Flatten()(x)
    #MYEDIT: concatenating the sun_pos and sampi
    x = keras.layers.Concatenate(axis=1)([x, x2_in, x3_in])

    x = keras.layers.Dense(dense_size, activation='relu')(x)
    x = keras.layers.Dropout(drop_rate)(x)
    x = keras.layers.Dense(dense_size, activation='relu')(x)
    x = keras.layers.Dropout(drop_rate)(x)

    ## regression to prediction target
    y_out = keras.layers.Dense(units=1)(x)

    # construct the model MYEDIT: adding the sun_pos as input
    model = keras.Model(inputs=[x_in, x2_in, x3_in],outputs=y_out)

    return model

# show model architecture
sunset_model().summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv2d_4 (Conv2D)              (None, 64, 64, 24)   672         ['input_7[0][0]']                
                                                                                                  
 batch_normalization_4 (BatchNo  (None, 64, 64, 24)  96          ['conv2d_4[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 max_pooling2d_4 (MaxPooling2D)  (None, 32, 32, 24)  0           ['batch_normalization_4[0][

### Model testing

In [ ]:
# load testing data
times_test = np.load(os.path.join(data_folder,"times_test.npy"),allow_pickle=True)
print("times_test.shape:", times_test.shape)

with h5py.File(data_path,'r') as f:

    # read in the data
    images_log_test = f['test']['images_log'][...]
    pv_log_test = f['test']['pv_log'][...]
    pv_pred_test = f['test']['pv_pred'][...]

# process image data
images_log_test = np.transpose(images_log_test,axes=[0,2,3,1,4])
images_log_test = np.reshape(images_log_test, [images_log_test.shape[0],images_log_test.shape[1],images_log_test.shape[2],-1])
images_log_test = (images_log_test/255.0).astype('float32')
    
print("images_log_test.shape:",images_log_test.shape)
print("pv_log_test.shape:",pv_log_test.shape)
print("pv_pred_test.shape:",pv_pred_test.shape)

print('image ex ', images_log_test[0,0])
print('pv log ex ', pv_log_test[0,0])
print('pv pred ex ', pv_pred_test[0])

In [ ]:
# evaluate model on the test set and generate predictions
loss = np.zeros((num_fold,len(times_test)))
prediction = np.zeros((num_fold,len(times_test)))

for i in range(num_fold):
    # define model path
    print("loading repetition {0} model ...".format(i+1))
    model_path = os.path.join(output_folder,'repetition_'+str(i+1),'best_model_repitition_'+str(i+1)+'.h5')
    # load the trained model
    model = keras.models.load_model(model_path)
    
    # model evaluation
    print("evaluating performance for the model".format(i+1))
    loss[i] = model.evaluate(x=[images_log_test,pv_log_test], y=pv_pred_test, batch_size=200, verbose=1)
    
    # generate prediction
    print("generating predictions for the model".format(i+1))
    prediction[i] = np.squeeze(model.predict([images_log_test,pv_log_test], batch_size=200, verbose=1))

# saving predictions from each model
np.save(os.path.join(output_folder,'test_predictions.npy'),prediction)

# using the ensemble mean of the 10 models as the final prediction 
print('-'*50)
print("model ensembling ...")
prediction_ensemble = np.mean(prediction,axis=0)
loss_ensemble = np.sqrt(np.mean((prediction_ensemble-pv_pred_test)**2))
print("the test set RMSE is {0:.3f} for the ensemble model".format(loss_ensemble))

In [ ]:
# formulate sunny and cloudy test days
sunny_dates = [(2017,9,15),(2017,10,6),(2017,10,22),
               (2018,2,16),(2018,6,12),(2018,6,23),
               (2019,1,25),(2019,6,23),(2019,7,14),(2019,10,14)]
cloudy_dates = [(2017,6,24),(2017,9,20),(2017,10,11),
                (2018,1,25),(2018,3,9),(2018,10,4),
                (2019,5,27),(2019,6,28),(2019,8,10),(2019,10,19)]
sunny_dates_test = [datetime.date(day[0],day[1],day[2]) for day in sunny_dates]
cloudy_dates_test = [datetime.date(day[0],day[1],day[2]) for day in cloudy_dates]

dates_test = np.asarray([times.date() for times in times_test])

## generate mask for the sunny days
mask = np.zeros(len(pv_pred_test),dtype=bool)
for i in sunny_dates_test:
    mask[np.where(dates_test==i)[0]]=1

## apply the mask to the dataset
times_test_sunny = times_test[mask]
pv_pred_test_sunny = pv_pred_test[mask]
pv_log_test_sunny = pv_log_test[mask]
prediction_ensemble_sunny = prediction_ensemble[mask]
print("times_test_sunny.shape:",times_test_sunny.shape)

times_test_cloudy = times_test[~mask]
pv_pred_test_cloudy = pv_pred_test[~mask]
pv_log_test_cloudy = pv_log_test[~mask]
prediction_ensemble_cloudy = prediction_ensemble[~mask]
print("times_test_cloudy.shape:",times_test_cloudy.shape)

In [ ]:
# RMSE for sunny and cloudy days individually
rmse_sunny = np.sqrt(np.mean(np.square((prediction_ensemble_sunny-pv_pred_test_sunny))))
rmse_cloudy = np.sqrt(np.mean(np.square((prediction_ensemble_cloudy-pv_pred_test_cloudy))))
rmse_overall = np.sqrt((rmse_sunny**2*len(pv_pred_test_sunny)+rmse_cloudy**2*len(pv_pred_test_cloudy))/(len(pv_pred_test)))

print("test set sunny days RMSE: {0:.3f}".format(rmse_sunny))
print("test set cloudy days RMSE: {0:.3f}".format(rmse_cloudy))
print("test set overall RMSE: {0:.3f}".format(rmse_overall))

In [ ]:
# MAE for sunny and cloudy days individually
mae_sunny = np.mean(np.abs((prediction_ensemble_sunny-pv_pred_test_sunny)))
mae_cloudy = np.mean(np.abs((prediction_ensemble_cloudy-pv_pred_test_cloudy)))
mae_overall = (mae_cloudy*len(pv_pred_test_cloudy) + mae_sunny*len(pv_pred_test_sunny))/(len(pv_pred_test))

print("test set sunny days MAE: {0:.3f}".format(mae_sunny))
print("test set cloudy days MAE: {0:.3f}".format(mae_cloudy))
print("test set overall MAE: {0:.3f}".format(mae_overall))

In [ ]:
# forecast skill for sunny and cloudy days individually
## calculate the rmse of persistent model
from Relative_op_func import *

ntimes = times_test.shape[0]
per_prediction = np.zeros(ntimes)
for i in range(ntimes):
    CSI_cur, P_theo_cur = Relative_output(times_test[i],pv_log_test[i][0])
    CSI, P_theo_pred = Relative_output(times_test[i]+datetime.timedelta(minutes=15),pv_pred_test[i])
    per_prediction[i] = CSI_cur*P_theo_pred

per_prediction_sunny = per_prediction[mask]
per_prediction_cloudy = per_prediction[~mask]

per_rmse_sunny = np.sqrt(np.mean(np.square((per_prediction_sunny-pv_pred_test_sunny))))
per_rmse_cloudy = np.sqrt(np.mean(np.square((per_prediction_cloudy-pv_pred_test_cloudy))))
per_rmse_overall = np.sqrt((per_rmse_sunny**2*len(pv_pred_test_sunny)+per_rmse_cloudy**2*len(pv_pred_test_cloudy))/(len(pv_pred_test)))
print("test set sunny days RMSE persistence: {0:.3f}".format(per_rmse_sunny))
print("test set cloudy days RMSE persistence: {0:.3f}".format(per_rmse_cloudy))
print("test set overall RMSE persistence: {0:.3f}".format(per_rmse_overall))

## calculate the forecasting skills
forecast_skill_sunny = 1 - rmse_sunny/per_rmse_sunny
forecast_skill_cloudy = 1 - rmse_cloudy/per_rmse_cloudy
forecast_skill_overall = 1 - rmse_overall/per_rmse_overall
print("test set sunny days forecast skill: {0:.2f}%".format(forecast_skill_sunny*100))
print("test set cloudy days forecast skill: {0:.2f}%".format(forecast_skill_cloudy*100))
print("test set overall forecast skill: {0:.2f}%".format(forecast_skill_overall*100))

In [ ]:
# visualization of forecast predictions
dates_test = np.array([dtinfo.date() for dtinfo in times_test])
hours_test = np.array([dtinfo.time() for dtinfo in times_test])

f,axarr = plt.subplots(10,2,sharex=False, sharey = True)
xfmt = mdates.DateFormatter('%H')
fmt_date = datetime.date(2000,1,1)

green = '#8AB8A7'
red = '#B83A4B'
blue = '#67AFD2'
grey =  '#B6B1A9'

for i,date in enumerate(sunny_dates_test):
    ax = axarr[i,0]
    date_mask = (dates_test == date)
    hours_xaxis= [datetime.datetime.combine(fmt_date, hour) for hour in hours_test[date_mask]] 
    
    rmse = np.sqrt(np.mean(np.square((pv_pred_test[date_mask]-prediction_ensemble[date_mask]))))
    mae = np.mean(np.abs((pv_pred_test[date_mask]-prediction_ensemble[date_mask])))
    per_rmse = np.sqrt(np.mean(np.square((pv_pred_test[date_mask]-per_prediction[date_mask]))))
    fs = (1 - rmse/per_rmse)*100
    
    ax.plot(hours_xaxis, pv_pred_test[date_mask], linewidth = 1,color=grey)
    ax.fill_between(hours_xaxis, 0, pv_pred_test[date_mask], color=grey, alpha=.2, label = 'Ground truth')
    ax.plot(hours_xaxis, prediction_ensemble[date_mask],linewidth = 1,label = 'SUNSET forecast',color=red,markerfacecolor="None")
    ax.set_ylabel('PV output (kW)')
    ax.xaxis.set_major_formatter(xfmt)
    ax.text(0.75,0.85,'Sunny_'+str(i+1), transform=ax.transAxes)
    ax.text(0.05,0.65,"RMSE: {0:.2f}\nMAE: {1:.2f}\nFS: {2:.2f}%".format(rmse,mae,fs),transform=ax.transAxes)

for i,date in enumerate(cloudy_dates_test):
    ax = axarr[i,1]
    date_mask = (dates_test == date)
    hours_xaxis= [datetime.datetime.combine(fmt_date, hour) for hour in hours_test[date_mask]] 
    
    rmse = np.sqrt(np.mean(np.square((pv_pred_test[date_mask]-prediction_ensemble[date_mask]))))
    mae = np.mean(np.abs((pv_pred_test[date_mask]-prediction_ensemble[date_mask])))
    per_rmse = np.sqrt(np.mean(np.square((pv_pred_test[date_mask]-per_prediction[date_mask]))))
    fs = (1 - rmse/per_rmse)*100
    
    ax.plot(hours_xaxis, pv_pred_test[date_mask], linewidth = 1,color=grey)
    ax.fill_between(hours_xaxis, 0, pv_pred_test[date_mask], color=grey, alpha=.2, label = 'Ground truth')
    ax.plot(hours_xaxis, prediction_ensemble[date_mask],linewidth = 1,label = 'SUNSET forecast',color=red,markerfacecolor="None")
    ax.set_ylabel('PV output (kW)')
    ax.xaxis.set_major_formatter(xfmt)
    ax.text(0.75,0.85,'Cloudy_'+str(i+1), transform=ax.transAxes)
    ax.text(0.05,0.65,"RMSE: {0:.2f}\nMAE: {1:.2f}\nFS: {2:.2f}%".format(rmse,mae,fs),transform=ax.transAxes)

    
axarr[0,0].set_ylim(0, 30)
axarr[0,0].legend(bbox_to_anchor= [1.15,1.3], loc = 'upper center', ncol = 3)
axarr[-1,0].set_xlabel('Hour of day')
axarr[-1,1].set_xlabel('Hour of day')

f.set_size_inches(10,25)    
plt.show()  